In [1]:
import pandas as pd
import numpy as np
import datetime
import statsmodels as sm
import matplotlib.pyplot as plt
import seaborn as sns 
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import adfuller
import matplotlib

%matplotlib inline

In [2]:
df_raw = pd.read_csv('zillow_data.csv')
df_raw.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


In [3]:
#zipcode database
df_Zipcode = pd.read_csv('zip_code_database.csv')

In [4]:
df_raw.rename(columns={'RegionName': 'Zipcode'}, inplace = True)


In [5]:
df_Zipcode.rename(columns={'zip':'Zipcode'}, inplace = True)
df_Zipcode = df_Zipcode[['Zipcode', 'latitude', 'longitude', 'irs_estimated_population' ]]
df_Zipcode.head()

,Zipcode,latitude,longitude,irs_estimated_population
0,501,40.81,-73.04,562
1,544,40.81,-73.04,0
2,601,18.16,-66.72,0
3,602,18.38,-67.18,0
4,603,18.43,-67.15,0


In [6]:
def get_datetimes(df):
    """
    Takes a dataframe:
    returns only those column names that can be converted into datetime objects 
    as datetime objects.
    NOTE number of returned columns may not match total number of columns in passed dataframe
    """
    
    return pd.to_datetime(df.columns.values[7:], format='%Y-%m')

In [7]:
pd.melt(df_raw)

,variable,value
0,RegionID,84654
1,RegionID,90668
2,RegionID,91982
3,RegionID,84616
4,RegionID,93144
...,...,...
4004651,2018-04,209300
4004652,2018-04,225800
4004653,2018-04,133400
4004654,2018-04,664400


In [15]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

# NOTE: if you visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!

In [16]:
get_datetimes(df_raw)

DatetimeIndex(['1996-04-01', '1996-05-01', '1996-06-01', '1996-07-01',
               '1996-08-01', '1996-09-01', '1996-10-01', '1996-11-01',
               '1996-12-01', '1997-01-01',
               ...
               '2017-07-01', '2017-08-01', '2017-09-01', '2017-10-01',
               '2017-11-01', '2017-12-01', '2018-01-01', '2018-02-01',
               '2018-03-01', '2018-04-01'],
              dtype='datetime64[ns]', length=265, freq=None)

In [17]:
df = df_raw.merge(df_Zipcode, on = 'Zipcode')
df.head()

,RegionID,Zipcode,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,latitude,longitude,irs_estimated_population
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1009600,1013300,1018700,1024400,1030700,1033800,1030600,41.94,-87.65,54760
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,314100,315000,316600,318100,319600,321100,321800,33.17,-96.70,56810
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,320400,320800,321200,321200,323000,326900,329900,29.74,-95.83,125330
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1291500,1296600,1299000,1302700,1306400,1308500,1307000,41.92,-87.65,52800
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,120300,120300,120300,120300,120500,121000,121500,31.78,-106.30,99590


In [18]:
#Dividing zipcodes into region 
df_East = df_raw[df_raw.Zipcode < 50000]
df_Central = df_raw[(df_raw.Zipcode >= 50000) & (df_raw.Zipcode < 80000) ]
df_West = df[(df.Zipcode >= 80000)]
#West coast** 
df_West.head()

,RegionID,Zipcode,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,latitude,longitude,irs_estimated_population
9,97564,94109,San Francisco,CA,San Francisco,San Francisco,10,766000.0,771100.0,776500.0,...,3799700,3793900,3778700,3770800,3763100,3779800,3813500,37.79,-122.42,39680
15,96107,90250,Hawthorne,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,16,152500.0,152600.0,152600.0,...,594700,598500,601300,602800,606100,612400,616200,33.91,-118.34,85010
27,97771,94565,Pittsburg,CA,San Francisco,Contra Costa,28,139200.0,138300.0,137500.0,...,405400,408600,410900,413700,417900,424300,430100,38.01,-121.91,89590
30,96027,90046,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,31,340600.0,341700.0,343000.0,...,1903900,1907500,1922100,1952400,1974500,1975900,1966900,34.11,-118.36,35600
56,94748,85032,Phoenix,AZ,Phoenix,Maricopa,57,95400.0,95800.0,96200.0,...,259500,260700,261900,263100,264100,265400,266400,33.62,-112.00,57380


In [19]:
cali_df = df_raw[(df_raw['State'] == 'CA')]
cali_df.info()
tx_df = df_raw[(df_raw['State'] == 'TX')]
tx_df.info()
fl_df = df_raw[(df_raw['State'] == 'FL')]
fl_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1224 entries, 9 to 14712
Columns: 272 entries, RegionID to 2018-04
dtypes: float64(219), int64(49), object(4)
memory usage: 2.5+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 989 entries, 1 to 14695
Columns: 272 entries, RegionID to 2018-04
dtypes: float64(219), int64(49), object(4)
memory usage: 2.1+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 785 entries, 11 to 14600
Columns: 272 entries, RegionID to 2018-04
dtypes: float64(219), int64(49), object(4)
memory usage: 1.6+ MB


In [21]:
cali_df.shape
cali_df.head()

,RegionID,Zipcode,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
9,97564,94109,San Francisco,CA,San Francisco,San Francisco,10,766000.0,771100.0,776500.0,...,3767700,3763900,3775000,3799700,3793900,3778700,3770800,3763100,3779800,3813500
15,96107,90250,Hawthorne,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,16,152500.0,152600.0,152600.0,...,579300,585700,590900,594700,598500,601300,602800,606100,612400,616200
27,97771,94565,Pittsburg,CA,San Francisco,Contra Costa,28,139200.0,138300.0,137500.0,...,394900,398400,401600,405400,408600,410900,413700,417900,424300,430100
30,96027,90046,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,31,340600.0,341700.0,343000.0,...,1839800,1861100,1888600,1903900,1907500,1922100,1952400,1974500,1975900,1966900
64,97711,94501,Alameda,CA,San Francisco,Alameda,65,222400.0,222300.0,222400.0,...,965100,975000,987800,1000000,1009900,1021000,1032400,1042000,1053000,1059700


## California (West Coast)

In [12]:
cali_df.isnull().sum()

RegionID     0
Zipcode      0
City         0
State        0
Metro       42
            ..
2017-12      0
2018-01      0
2018-02      0
2018-03      0
2018-04      0
Length: 272, dtype: int64

In [13]:
cali_df.isna().sum().sum()

5658

In [22]:
cali_df.Metro.unique()

array(['San Francisco', 'Los Angeles-Long Beach-Anaheim', 'Sacramento',
       'Napa', 'San Diego', 'Fresno', 'Vallejo', 'Riverside', 'Ventura',
       'San Jose', 'Bakersfield', 'Visalia', 'Hanford', 'Santa Cruz',
       'Modesto', 'Redding', 'Santa Maria-Santa Barbara', 'Stockton',
       'Salinas', 'San Luis Obispo', 'Santa Rosa', 'Chico', 'El Centro',
       'Yuba City', 'Merced', 'Ukiah', 'Truckee', 'Madera', 'Eureka',
       'Susanville', nan, 'Clearlake'], dtype=object)

In [29]:
cali_df['Metro'].fillna('empty', inplace = True)
cali_df.isnull().sum()

/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


RegionID     0
Zipcode      0
City         0
State        0
Metro        0
            ..
2018-02      0
2018-03      0
2018-04      0
ROI_5       14
ROI_3        0
Length: 274, dtype: int64

In [30]:
cali_df.iloc[:,7:].isnull().sum()

1996-04    36
1996-05    36
1996-06    36
1996-07    36
1996-08    36
           ..
2018-02     0
2018-03     0
2018-04     0
ROI_5      14
ROI_3       0
Length: 267, dtype: int64

In [32]:
cali_df.dropna(inplace=True, subset=list(cali_df.columns[216:]))

<ipython-input-32-4c2018dc1e43>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cali_df.dropna(inplace=True, subset=list(cali_df.columns[216:]))


In [23]:
## 5 year ROI
cali_df['ROI_5'] = round((cali_df['2018-04'] - cali_df['2013-01'])/ cali_df['2013-01'],4)
## 3 year ROI, this metric should be the metric used
cali_df['ROI_3'] = round((cali_df['2018-04'] - cali_df['2015-01'])/ cali_df['2015-01'],4)

<ipython-input-23-e993a3714d24>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cali_df['ROI_5'] = round((cali_df['2018-04'] - cali_df['2013-01'])/ cali_df['2013-01'],4)
<ipython-input-23-e993a3714d24>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cali_df['ROI_3'] = round((cali_df['2018-04'] - cali_df['2015-01'])/ cali_df['2015-01'],4)


In [25]:
cali_df['ROI_5']

9        0.3857
15       0.6776
27       1.2530
30       0.7865
64       0.7501
          ...  
14683    0.4891
14690    0.4931
14692    0.4047
14709    0.8307
14712    0.4653
Name: ROI_5, Length: 1224, dtype: float64

In [26]:
cali_df['ROI_3']

9        0.1130
15       0.3597
27       0.4745
30       0.4055
64       0.3942
          ...  
14683    0.1660
14690    0.1373
14692    0.2159
14709    0.5079
14712    0.2911
Name: ROI_3, Length: 1224, dtype: float64

In [28]:
cali_df.info

<bound method DataFrame.info of        RegionID  Zipcode               City State  \
9         97564    94109      San Francisco    CA   
15        96107    90250          Hawthorne    CA   
27        97771    94565          Pittsburg    CA   
30        96027    90046        Los Angeles    CA   
64        97711    94501            Alameda    CA   
...         ...      ...                ...   ...   
14683     97304    93517         Bridgeport    CA   
14690     98404    95728            Truckee    CA   
14692     98245    95497          Annapolis    CA   
14709     96805    92322          Crestline    CA   
14712     96822    92341  Green Valley Lake    CA   

                                Metro      CountyName  SizeRank   1996-04  \
9                       San Francisco   San Francisco        10  766000.0   
15     Los Angeles-Long Beach-Anaheim     Los Angeles        16  152500.0   
27                      San Francisco    Contra Costa        28  139200.0   
30     Los Angeles-Long

In [ ]:
#RWM
avg = 0
std = 10 
n_pts = 1000
date_vals = pd.date_range(starts = '04/01/1996')

In [ ]:
# def melt_data(df):
#     """
#     Takes the zillow_data dataset in wide form or a subset of the zillow_dataset.  
#     Returns a long-form datetime dataframe 
#     with the datetime column names as the index and the values as the 'values' column.
    
#     If more than one row is passes in the wide-form dataset, the values column
#     will be the mean of the values from the datetime columns in all of the rows.
#     """
    
#     melted = pd.melt(df, id_vars=['RegionName', 'RegionID', 'SizeRank', 'City', 'State', 'Metro', 'CountyName'], var_name='time')
#     melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
#     melted = melted.dropna(subset=['value'])
#     return melted.groupby('time').aggregate({'value':'mean'})